<a href="https://colab.research.google.com/github/Yuns-u/KoBERT_emotion_Classifier/blob/main/KoBERT_Emotion_Classifier_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT를 위한 환경 설정

In [ ]:
#KoBERT를 사용하기 위한 라이브러리 설정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

# AI Hub에서 받아온 데이터 전처리하기

- 공포 : 5468 
- 놀람 : 5898 
- 분노 : 5665
- 슬픔 : 5267
- 중립 : 4830
- 행복 : 6037
- 혐오 : 5429

총 38594개의 말뭉치이다.

위처럼 일곱가지의 감정으로 분류되어 있지만 각 감정의 데이터 수 차이가 있다.
또한 감정의 수가 7가지나 되기 때문에 목적에 맞게 긍정(1), 부정(0), 중립(0.5) 정도로 분류하면 1:다 대화에서의 감정을 파악하는 데에 도움이 될 것이라고 생각한다.

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
myfile = files.upload()

In [ ]:
#pd.read_csv로 csv파일 불러오기
df = pd.read_csv('korean_oneoff_chats.csv')
df

In [ ]:
#결측치 확인
df.isna().sum()

## Emotion별 특징 분석하기
행복은 긍정이고 혐오는 부정인 것은 직관적으로 알 수 있지만 놀람, 공포 등 긍정인지 부정인지 명확하게 구분하기 어려운 감정말뭉치들이 있는 것으로 보인다.

해당 말뭉치가 단발성이라는 점에서 반어법 등의 유무를 살펴볼 수는 없었다. 
- 긍정: 행복
- 중립: 중립
- 부정: 혐오

공포, 놀람, 분노, 슬픔은 어떻게 나눌 수 있는가?

In [ ]:
df['Emotion'].unique()

일단 자의로 감정을 분석하는 것은 오히려 모델의 성능을 해칠 수 있으므로 7가지 감정에 대해 분류를 해보도록 하겠다.

- 혐오 : 0
- 분노 : 1
- 공포 : 2
- 중립 : 3
- 슬픔 : 4
- 놀람 : 5 
- 행복 : 6


In [ ]:
df.loc[(df['Emotion']=='혐오'), 'Emotion'] = 0 #혐오를 0으로 표현
df.loc[(df['Emotion']=='분노'), 'Emotion'] = 1 #분노를 1로 표현
df.loc[(df['Emotion']=='공포'), 'Emotion'] = 2 #공포를 2로 표현
df.loc[(df['Emotion']=='중립'), 'Emotion'] = 3 #중립을 3으로 표현
df.loc[(df['Emotion']=='슬픔'), 'Emotion'] = 4 #슬픔을 4로 표현
df.loc[(df['Emotion']=='놀람'), 'Emotion'] = 5 #놀람을 5로 표현
df.loc[(df['Emotion']=='행복'), 'Emotion'] = 6 #행복을 6으로 표현

In [ ]:
df.sample(10)

In [ ]:
df.describe()

In [ ]:
#각 행의 데이터를 리스트로 묶어주고 하나의 리스트에 넣기
data_list = []

for q, label in zip(df['Sentence'], df['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])

# 데이터셋 나누기

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
train_li, test_li = train_test_split(data_list, test_size=0.2, random_state=0)

In [ ]:
print(len(train_li))
print(len(test_li))

# KoBERT 입력 데이터로 데이터셋 만들기

In [ ]:
#BERT 모델에 들어가기 위한 데이터셋을 만들어주는 사용자 클래스 만들기
class BERTdata(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return(len(self.labels))

In [ ]:
# KoBERT의 파라미터 세팅
max_len = 64             #텍스트 데이터 최대 길이
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5           #반복학습 수
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

# 토큰화

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
#위의 클래스를 통해 데이터프레임에 있던 데이터를 토큰화해주기
train_data = BERTdata(train_li, 0, 1, tok, max_len, True, False)
test_data = BERTdata(test_li, 0, 1, tok, max_len, True, False)

In [ ]:
train_data[0]

In [ ]:
#torch형식의 데이터셋으로 만들어주어 koBERT에 학습시킬 수 있는 형태로 만들어주기

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=5)

# 감정 분류 KoBERT 학습 모델 만들기

# 추가사항

- 1:다 채팅의 경우 이모티콘도 감정 표현이 될 것 같은데... 하트 등의 이모티콘을 행복으로 표현한다면 좀 더 정확한 측정이 가능하지 않을까?